In [102]:
pip install dask

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import glob
import geopandas as gpd
import pandas as pd
import ast
import json
from shapely.geometry import Point, LineString
import dask
import dask.dataframe as dd
from dask.distributed import Client

client = dask.distributed.Client()

OSError: could not find or load spatialindex_c-64.dll

In [3]:
csv_file = r'F:\\Safegraph\\Neigborhood\\patterns\\2021\\05\\04\\y=2021\\m=4\\part-00000-tid-1265847720957935305-ce8e0626-c194-4c9d-82d2-56460c6b1370-446576-1-c000.csv.gz'

csv_file = r"H:\Safegraph\Weekly Places Patterns v2 (until 2020-06-15)\main-file\2019-02-11-weekly-patterns.csv.gz"

csv_files = r"H:\Safegraph\Weekly Places Patterns (for data from 2020-11-30 to Present)\patterns\2021\05\26\18\*.gz"

In [9]:
csv_files = r"H:\Safegraph\Weekly Places Patterns (for data from 2020-11-30 to Present)\patterns\2021\05\26\18\*.gz"

csv_file = r'F:\\Safegraph\\Neigborhood\\patterns\\2021\\05\\04\\y=2021\\m=4\\part-00000-tid-1265847720957935305-ce8e0626-c194-4c9d-82d2-56460c6b1370-446576-1-c000.csv.gz'


# ddf = dd.read_csv(csv_file, blocksize=None, encoding='utf-8').compute(scheduler='processes')
df = pd.read_csv(csv_file,  encoding='utf-8') 

In [91]:
%%time
csv_files = r"H:\Safegraph\Weekly Places Patterns (for data from 2020-11-30 to Present)\patterns\2021\05\26\18\*.csv"
df = dd.read_csv(csv_files).compute()

Wall time: 1min 9s


In [92]:
ddf = dd.from_pandas(df, npartitions=12)
type(ddf)

dask.dataframe.core.DataFrame

In [5]:
csv_files = r"H:\Safegraph\Weekly Places Patterns (for data from 2020-11-30 to Present)\patterns\2021\05\26\18\*.gz"
%%time
ddf = dd.read_csv(csv_files, blocksize=None).compute()

Wall time: 1min 42s


In [11]:
len(ddf)

4428203

In [60]:
client = dask.distributed.Client()

AttributeError: module 'dask' has no attribute 'distributed'

In [12]:
len(ddf["visitor_home_cbgs"].unique())

2822667

In [93]:
import json
import dask


def unfold_row_dict1(row,):
    placekey = row["placekey"]
    a_dict = json.loads(row["visitor_home_cbgs"])
    return str(list(zip([placekey] * len(a_dict.keys()), a_dict.keys(), a_dict.values())))
    
def unfold_row_dict(row, result_list):
    placekey = row["placekey"]
    a_dict = json.loads(ddf.iloc[1]["visitor_home_cbgs"])
    result_list += list(zip([placekey] * len(a_dict.keys()), a_dict.keys(), a_dict.values()))

pair_list = []
# unfold_row_dict(ddf.iloc[1], pair_list)

# ddf.iloc[:].apply(unfold_row_dict, args=(pair_list,), axis=1).compute()
s_ddf = ddf.sample(1)
print(len(s_ddf))
print(type(s_ddf))
print(type(ddf))

s_ddf["pairs"] = s_ddf.apply(unfold_row_dict1, axis=1).compute()


s_ddf["pairs"]

C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\dask\dataframe\core.py:1393: UserWarning: sample does not support the number of sampled items parameter, 'n'. Please use the 'frac' parameter instead.
  warnings.warn(msg)


4428203
<class 'dask.dataframe.core.DataFrame'>
<class 'dask.dataframe.core.DataFrame'>


ValueError: Metadata inference failed in `apply`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
JSONDecodeError('Expecting value: line 1 column 1 (char 0)')

Traceback:
---------
  File "C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\dask\dataframe\utils.py", line 176, in raise_on_meta_error
    yield
  File "C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\dask\dataframe\core.py", line 5571, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\dask\utils.py", line 900, in __call__
    return getattr(obj, self.method)(*args, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\pandas\core\frame.py", line 7768

In [175]:
def unfold_row_dict1(row):
    placekey = row["placekey"]
    cell_text = row["visitor_home_cbgs"]
    # print(cell_text)
    if cell_text == "":
        return ""
    a_dict = json.loads(cell_text)
    # print(a_dict)
    return list(zip([placekey] * len(a_dict.keys()), a_dict.keys(), a_dict.values()))

def unfold_row_dict(row, result_list):
    placekey = row["placekey"]
    a_dict = json.loads(row["visitor_home_cbgs"])
    result_list += list(zip([placekey] * len(a_dict.keys()), a_dict.keys(), a_dict.values()))
    
# s_ddf = df.sample(len(df))
s_ddf = df.sample(10000)
# s_ddf = ddf.sample(0.0006)
print(len(s_ddf))
print(type(s_ddf))


pair_list = []
# df["pairs"] = df.apply(unfold_row_dict1, axis=1)
df.apply(unfold_row_dict, args=(pair_list,), axis=1)
len(pair_list)

10000
<class 'pandas.core.frame.DataFrame'>


29223229

In [177]:
pair_list_df = pd.DataFrame(pair_list)

In [184]:
pair_list_df.columns = ["placekey", "visitor_home_cbgs", "visits"]

In [191]:
pair_list_df

,placekey,visitor_home_cbgs,visits
0,zzz-222@63j-f68-6rk,391130404053,7
1,zzz-222@63j-f68-6rk,391130403051,5
2,zzz-222@63j-f68-6rk,391130401031,5
3,zzz-222@63j-f68-6rk,390572403014,5
4,zzz-222@63j-f68-6rk,391130404013,5
...,...,...,...
29223224,zzw-223@5pw-6qv-249,170310804002,4
29223225,zzw-223@5pw-6qv-249,170318083012,4
29223226,224-222@66b-8ns-33q,360191018004,4
29223227,zzw-224@63f-kkm-q2k,371830529042,4


In [195]:
saved_path = os.path.dirname(csv_files)
saved_name = os.path.join(saved_path, "edges.csv")
%time
pair_list_df.to_csv(saved_name, index=False)

Wall time: 0 ns


In [197]:
visits_sum_df = pair_list_df.groupby("visitor_home_cbgs")["visits"].sum().reindex()

In [199]:
visits_sum_df.columns = ["sum"]

In [204]:
visits_sum_df.sort_values()

visitor_home_cbgs
CA:62080022         4
CA:35204172         4
CA:24390210         4
CA:24390204         4
CA:24390200         4
                ...  
120950167301    28901
481576732002    30302
481576729001    34118
120950171032    35368
481576731011    43024
Name: visits, Length: 259230, dtype: int64

# Draw maps

In [174]:
df.iloc[-4000]["visitor_home_cbgs"]

'{"060650430061":4,"060650432711":4,"060650430012":4}'

In [67]:
client = dask.distributed.Client()

C:\ProgramData\Anaconda3\envs\gdal_env\lib\site-packages\distributed\node.py:164: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50043 instead
  expected, actual


In [43]:
ddf.dtypes

placekey                      object
safegraph_place_id            object
parent_placekey               object
parent_safegraph_place_id     object
location_name                 object
street_address                object
city                          object
region                        object
postal_code                   object
iso_country_code              object
safegraph_brand_ids           object
brands                        object
date_range_start              object
date_range_end                object
raw_visit_counts               int64
raw_visitor_counts             int64
visits_by_day                 object
visits_by_each_hour           object
poi_cbg                       object
visitor_home_cbgs             object
visitor_daytime_cbgs          object
visitor_country_of_origin     object
distance_from_home           float64
median_dwell                 float64
bucketed_dwell_times          object
related_same_day_brand        object
related_same_week_brand       object
d

In [40]:
def unfold_dict(a_dict):
    reutrn zip(a_dict.keys(), a_dict.values())

SyntaxError: invalid syntax (<ipython-input-40-423c42e73cfc>, line 2)

In [23]:
json.loads(ddf.iloc[1]["visitor_home_cbgs"])

{'391130404053': 7,
 '391130403051': 5,
 '391130401031': 5,
 '390572403014': 5,
 '391130404013': 5,
 '391130401021': 4,
 '391650305041': 4,
 '391650325012': 4,
 '391093801002': 4,
 '390610215053': 4,
 '391130404011': 4,
 '391650309004': 4,
 '390572201001': 4,
 '391130216012': 4,
 '391130403033': 4,
 '391130506003': 4,
 '390170110033': 4,
 '391130402012': 4,
 '390572701002': 4,
 '391650307002': 4,
 '391130404063': 4,
 '391650309001': 4,
 '121199112001': 4,
 '391130505043': 4,
 '391130404062': 4,
 '391130503011': 4,
 '391130403032': 4,
 '391130403063': 4}

In [127]:
ddf["device_type"].unique()

array(['{"android":0,"ios":0}', '{"android":53,"ios":71}',
       '{"android":22,"ios":63}', ..., '{"android":714,"ios":771}',
       '{"android":290,"ios":630}', '{"android":126,"ios":248}'],
      dtype=object)

# Extract targeted fips

In [60]:
def extract_neighorhood_pattern(df, fips):
    column_name = "device_home_areas"
    idxs  = df.apply(is_fips_in_row, args=(column_name, fips), axis=1)
    return df[idxs]
    
def is_fips_in_row(row, column_name, fips):
    return is_fips_in_dict(json.loads(row[column_name]), fips)
    
def is_fips_in_dict(poi_dict, fips):
    fips = str(fips)
    fips_len = len(fips)
    for key in poi_dict.keys():
        key = str(key)
        if key[:fips_len] == fips:
            # print("key:", key)
            return True
    return False

In [73]:
sampled_df = df.sample(10000)
column_name = "device_home_areas"
target_df = extract_neighorhood_pattern(sampled_df, "45")
print(len(target_df))

2250


In [89]:
sampled_df = df.sample(1)
sampled_df['stops_by_each_hour']

183574    [2,0,2,1,2,2,4,9,36,25,15,20,18,31,22,32,15,35...
Name: stops_by_each_hour, dtype: object

In [27]:
column_name = "device_home_areas"
t = extract_neighorhood_pattern(sampled_df, "45").iloc[0]

key: 450219704012


In [22]:
t_dict = json.loads(t['device_home_areas'])
t_dict.keys()


dict_keys(['131850115002', '131850116001', '131850114032', '131850107003', '131850115001', '131850115003', '131850114034', '131850114033', '131850114031', '131850116002', '131850102024', '131850102023', '131850114021', '131850114011', '131850102025', '131018802002', '131739502004', '131018802001', '131850108003', '131850114022', '120479602003', '130279603003', '131850106043', '131018801001', '131850102011', '131850101031', '130759604002', '130719709004', '120130103004', '130659701001', '131850102021', '131850112004', '131850101011', '131850104023', '131850102026', '120419501002', '131739502001', '130279606001', '131850111002', '131850104022', '130279603004', '131850102022', '132879703003', '131850106042', '131850103012', '131850103022', '131850113013', '130279604002', '130279605001', '131850112001', '121030250091', '131850101022', '131850102012', '481130031012', '120231105005', '131850104012', '130690108021', '120479602001', '120110309032', '131850103013', '131850105003', '131899505001

In [5]:
df.dtypes

area                                        int64
area_type                                  object
origin_area_type                           object
date_range_start                           object
date_range_end                             object
day_counts                                 object
raw_stop_counts                             int64
raw_device_counts                           int64
stops_by_day                               object
stops_by_each_hour                         object
device_home_areas                          object
weekday_device_home_areas                  object
weekend_device_home_areas                  object
breakfast_device_home_areas                object
lunch_device_home_areas                    object
afternoon_tea_device_home_areas            object
dinner_device_home_areas                   object
nightlife_device_home_areas                object
work_hours_device_home_areas               object
work_behavior_device_home_areas            object


In [101]:
columns = list(df.columns)
for c in columns:
    len_c = ''
    cell = df.iloc[11200][c]
    if str(cell)[0] == "[":
        a_list = ast.literal_eval(cell)
        len_c = len(a_list)
    if str(cell)[0] == "{":
        jdata = json.loads(cell)

        len_c = len(jdata.keys())
    print(f"{c}:, len: {len_c}     \n  {cell}\n")

safegraph_place_id:, len:      
  sg:738274ac416f48bb9ea90a611b8c9add

location_name:, len:      
  Dollar General

street_address:, len:      
  1807 N 195th Ave

city:, len:      
  Buckeye

region:, len:      
  AZ

postal_code:, len:      
  85396

iso_country_code:, len:      
  US

safegraph_brand_ids:, len:      
  SG_BRAND_a4efb9d8ddf0f71207d49f713c5bc1e9

brands:, len:      
  Dollar General

date_range_start:, len:      
  2019-02-11T00:00:00-07:00

date_range_end:, len:      
  2019-02-18T00:00:00-07:00

raw_visit_counts:, len:      
  12

raw_visitor_counts:, len:      
  12

visits_by_day:, len: 7     
  [0,0,2,2,3,3,2]

visits_by_each_hour:, len: 168     
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0

In [92]:
len(df)

220654

In [8]:
import json
jdata = json.loads(df.iloc[0]['device_home_areas'])
len(jdata.keys())

1342